In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# http://www.bcn.cat/tercerlloc/files/restaurants/opendatabcn_restaurants_restaurants-csv.csv
restaurants = pd.read_csv('/home/filip/Documents/UPC/SDM/Project/triphawk-knowledge-graph/data/restaurants.csv')
# http://www.bcn.cat/tercerlloc/files/opendatabcn_agenda-csv.csv
agenda = pd.read_csv('/home/filip/Documents/UPC/SDM/Project/triphawk-knowledge-graph/data/agenda.csv')

# Agenda

In [4]:
agenda = agenda[['register_id', 'name', 'modified', 'addresses_road_name', 'addresses_neighborhood_name', 'geo_epgs_4326_x', 'geo_epgs_4326_y']]
agenda = agenda.dropna()
agenda.rename(columns={'register_id': 'event_id', 'modified': 'date', 'addresses_road_name': 'address', 'addresses_neighborhood_name': 'neighborhood', 'geo_epgs_4326_x': 'lat', 'geo_epgs_4326_y': 'lon' }, inplace=True)
agenda.head()

,event_id,name,date,address,neighborhood,lat,lon
1,﻿144170134,Visita al Pailebot Santa Eulàlia al Moll de la...,2021-11-15T17:33:46.347033+01:00,Moll Bosch i Alsina,el Barri Gòtic,41.379108,2.181910
2,﻿99400579929,Casal d'estiu 'El mite d'Arkadios' per a infan...,2022-03-16T10:25:09.504265+01:00,Av Diagonal,el Fort Pienc,41.400891,2.177801
3,﻿99400580001,Campus Olímpia Específic de 'Pàdel amb piscina...,2022-04-22T10:47:17.340816+02:00,C Muntadas,Hostafrancs,41.376711,2.142398
4,﻿99400580179,Casal d'estiu 'Deixem petjada' per a infants i...,2022-03-28T11:10:29.715263+02:00,Rbla Badal,la Bordeta,41.371060,2.131944
5,﻿99400580319,Precampus Olímpia per a infants de 3 a 5 anys ...,2022-04-20T09:58:57.430273+02:00,C Sant Oleguer,el Raval,41.377311,2.170880


# Restaurants

In [5]:
restaurants = restaurants[['register_id', 'name', 'addresses_road_name', 'addresses_neighborhood_name', 'geo_epgs_4326_x', 'geo_epgs_4326_y']]
restaurants = restaurants.dropna()
restaurants.rename(columns={'register_id': 'restaurant_id', 'addresses_road_name': 'address', 'addresses_neighborhood_name': 'neighborhood', 'geo_epgs_4326_x': 'lat', 'geo_epgs_4326_y': 'lon' }, inplace=True)
restaurants.head()

,restaurant_id,name,address,neighborhood,lat,lon
0,﻿75990013952,Bar Restaurant Paco Alcalde,Carrer d'Emília Llorca Martín,la Barceloneta,41.377351,2.189766
1,﻿93057142057,Restaurant Rebost de la Plana,Pl Trilla,la Vila de Gràcia,41.403591,2.152498
2,﻿1271120255,Restaurant Coure,Ptge Marimon,Sant Gervasi - Galvany,41.394434,2.147899
3,﻿75990002937,Bar Restaurant A Canteira,Pg Sant Joan,el Camp d'en Grassot i Gràcia Nova,41.402235,2.165437
4,﻿168094730,Cerveseria Restaurant El Fraile *La Maquinista,Pg Potosí,el Bon Pastor,41.442562,2.198370


# Neighborhoods

In [7]:
nb1 = set(agenda.neighborhood)
nb2 = set(restaurants.neighborhood)
nbh_names = nb1.union(nb2)


In [8]:
id = [i+1 for i in range(len(nbh_names))]
d = {'neighborhood': list(nbh_names), 'nbh_id': id}
neighborhoods = pd.DataFrame(data=d)
neighborhoods.head()


,neighborhood,nbh_id
0,el Besòs i el Maresme,1
1,les Roquetes,2
2,el Guinardó,3
3,el Putxet i el Farró,4
4,la Prosperitat,5


# Relations

In [9]:
agenda_nbh = agenda.merge(neighborhoods, on='neighborhood')
agenda_nbh = agenda_nbh[['event_id', 'nbh_id']]

In [10]:
agenda_nbh

,event_id,nbh_id
0,﻿144170134,56
1,﻿1344102524,56
2,﻿2025121918,56
3,﻿99400612771,56
4,﻿99400655138,56
...,...,...
3076,﻿99400691157,8
3077,﻿99400692668,8
3078,﻿99400692675,8
3079,﻿99400688353,71


In [11]:
resto_nbh = restaurants.merge(neighborhoods, on='neighborhood')
resto_nbh = resto_nbh[['restaurant_id', 'nbh_id']]

# neo4j-import column names

In [13]:
agenda = agenda.drop('neighborhood', 1)
restaurants = restaurants.drop('neighborhood', 1)

/tmp/ipykernel_46783/1718831302.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  agenda = agenda.drop('neighborhood', 1)
/tmp/ipykernel_46783/1718831302.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  restaurants = restaurants.drop('neighborhood', 1)


In [14]:
agenda.rename(columns={'event_id': 'event_id:ID(Event)'}, inplace=True)

In [15]:
neighborhoods.rename(columns={'nbh_id': 'nbh_id:ID(Neighborhood)'}, inplace=True)

In [16]:
restaurants.rename(columns={'restaurant_id': 'restaurant_id:ID(Restaurant)'}, inplace=True)

In [17]:
resto_nbh.rename(columns={'restaurant_id': 'restaurant_id:START_ID(Restaurant)', 'nbh_id': 'nbh_id:END_ID(Neighborhood)'}, inplace=True)

In [18]:
agenda_nbh.rename(columns={'event_id': 'event_id:START_ID(Event)', 'nbh_id': 'nbh_id:END_ID(Neighborhood)'}, inplace=True)

In [ ]:
agenda.date = pd.to_datetime(agenda.date.apply(lambda x: x.split('T')[0]))

# TO_CSV

In [82]:
agenda.to_csv('./data/generated/event.csv', index=False)
restaurants.to_csv('./data/generated/restaurants.csv', index=False)
neighborhoods.to_csv('./data/generated/neighborhood.csv', index=False)
resto_nbh.to_csv('./data/generated/resto_nbh.csv', index=False)
agenda_nbh.to_csv('./data/generated/event_nbh.csv', index=False)
